In [2]:
import re
import os
from bisect import bisect
# import TibProcessing

In [5]:
dadrag = ['ཀུན', 'ཤིན', 'འོན']
# check for possible dadrag https://github.com/eroux/tibetan-spellchecker/blob/master/doc/second-suffix-da.md


In [6]:
def syl_info(syl):
    ''' returns the required info to part_agreement. either :
            - ད་དྲག
            - མཐའ་མེད
            - the syllable
    '''
    global dadrag
    
    mingzhi = get_mingzhi(syl)
    if syl in dadrag: 
        return 'dadrag'
    elif re.findall(mingzhi+'[ིེོུའསརྱྲླྷྭ]*$', syl) != []:
        return 'thame'
    else:
        return syl
    

In [10]:

def part_agreement(previous, particle):
    ''' 
        proposes the right particle according to the previous syllable. 
        In case of an invalid previous syllable, returns the particle preceded by *
        limitation : particle needs to be a separate syllabes. (the problems with wrong merged agreement will be flagged by get_mingzhi )
    '''
    if previous == 'dadrag' :
        final = 'ད་དྲག' 
    elif previous == 'thame':
        final = 'མཐའ་མེད'
    else:
        final = previous[-1]
        if final not in ['ག', 'ང', 'ད', 'ན', 'བ', 'མ', 'འ', 'ར', 'ལ', 'ས']:
            final = None
    
    if final != None:
        # added the ད་དྲག་ for all and the མཐའ་མེད་ for all in provision of all cases where an extra syllable is neede in verses
        # dadrag added according to Élie’s rules.
        dreldra = {'ད' : 'ཀྱི', 'བ' : 'ཀྱི', 'ས' : 'ཀྱི', 'ག' : 'གི', 'ང' : 'གི', 'ན' : 'གྱི', 'མ' : 'གྱི', 'ར' : 'གྱི', 'ལ' : 'གྱི', 'འ' : 'ཡི', 'མཐའ་མེད' : 'ཡི', 'ད་དྲག' : 'གྱི'}
        jedra = {'ད' : 'ཀྱིས', 'བ' : 'ཀྱིས', 'ས' : 'ཀྱིས', 'ག' : 'གིས', 'ང' : 'གིས', 'ན' : 'གྱིས', 'མ' : 'གྱིས', 'ར' : 'གྱིས', 'ལ' : 'གྱིས', 'འ' : 'ཡིས', 'མཐའ་མེད' : 'ཡིས', 'ད་དྲག' : 'གྱིས'}
        ladon = {'ག' : 'ཏུ', 'བ' : 'ཏུ', 'ང' : 'དུ', 'ད' : 'དུ', 'ན' : 'དུ', 'མ' : 'དུ', 'ར' : 'དུ', 'ལ' : 'དུ', 'འ' : 'རུ', 'ས' : 'སུ', 'མཐའ་མེད' : 'རུ', 'ད་དྲག' : 'ཏུ'}
        lhakce = {'ན' : 'ཏེ', 'ར' : 'ཏེ', 'ལ' : 'ཏེ', 'ས' : 'ཏེ', 'ད' : 'དེ', 'ག' : 'སྟེ', 'ང' : 'སྟེ', 'བ' : 'སྟེ', 'མ' : 'སྟེ', 'འ' : 'སྟེ', 'མཐའ་མེད' : 'སྟེ', 'ད་དྲག' : 'ཏེ'}
        gyendu = {'ག' : 'ཀྱང', 'ད' : 'ཀྱང', 'བ' : 'ཀྱང', 'ས' : 'ཀྱང', 'འ' : 'འང', 'ང' : 'ཡང', 'ན' : 'ཡང', 'མ' : 'ཡང', 'འ' : 'ཡང', 'ར' : 'ཡང', 'ལ' : 'ཡང', 'མཐའ་མེད' : 'ཡང', 'ད་དྲག' : 'ཀྱང'}
        jedu = {'ག' : 'གམ', 'ང' : 'ངམ', 'ད་དྲག' : 'ཏམ', 'ད' : 'དམ', 'ན' : 'ནམ', 'བ' : 'བམ', 'མ' : 'མམ', 'འ' : 'འམ', 'ར' : 'རམ', 'ལ' : 'ལམ', 'ས' : 'སམ', 'མཐའ་མེད' : 'འམ', 'ད་དྲག' : 'ཏམ'}
        dagdra = {'ག' : 'པ', 'ད' : 'པ', 'བ' : 'པ', 'ས' : 'པ', 'ན' : 'པ', 'མ' : 'པ', 'ག' : 'པོ', 'ད' : 'པོ', 'བ' : 'པོ', 'ས' : 'པོ', 'ན' : 'པོ', 'མ' : 'པོ', 'ང' : 'བ', 'འ' : 'བ', 'ར' : 'བ', 'ལ' : 'བ', 'ང' : 'བོ', 'འ' : 'བོ', 'ར' : 'བོ', 'ལ' : 'བོ', 'མཐའ་མེད' : 'བ', 'མཐའ་མེད' : 'བོ', 'ད་དྲག' : 'པ', 'ད་དྲག' : 'པོ'}
        lardu = {'ག' : 'གོ', 'ང' : 'ངོ', 'ད' : 'དོ', 'ན' : 'ནོ', 'བ' : 'བོ', 'མ' : 'མོ', 'འ' : 'འོ', 'ར' : 'རོ', 'ལ' : 'ལོ', 'ས' : 'སོ', 'མཐའ་མེད' : 'འོ', 'ད་དྲག' : 'ཏོ'}
        cing = {'ག' : 'ཅིང', 'ད' : 'ཅིང', 'བ' : 'ཅིང', 'ང' : 'ཞིང', 'ན' : 'ཞིང', 'མ' : 'ཞིང', 'འ' : 'ཞིང', 'ར' : 'ཞིང', 'ལ' : 'ཞིང', 'ས' : 'ཤིང', 'མཐའ་མེད' : 'ཞིང', 'ད་དྲག' : 'ཅིང'}
        ces = {'ག' : 'ཅེས', 'ད' : 'ཅེས', 'བ' : 'ཅེས', 'ང' : 'ཞེས', 'ན' : 'ཞེས', 'མ' : 'ཞེས', 'འ' : 'ཞེས', 'ར' : 'ཞེས', 'ལ' : 'ཞེས', 'ས' : 'ཞེས', 'མཐའ་མེད' : 'ཞེས', 'ད་དྲག' : 'ཅེས'}
        ceo = {'ག' : 'ཅེའོ', 'ད' : 'ཅེའོ', 'བ' : 'ཅེའོ', 'ང' : 'ཞེའོ', 'ན' : 'ཞེའོ', 'མ' : 'ཞེའོ', 'འ' : 'ཞེའོ', 'ར' : 'ཞེའོ', 'ལ' : 'ཞེའོ', 'ས' : 'ཤེའོ', 'མཐའ་མེད' : 'ཞེའོ', 'ད་དྲག' : 'ཅེའོ', }
        cena = {'ག' : 'ཅེ་ན', 'ད' : 'ཅེ་ན', 'བ' : 'ཅེ་ན', 'ང' : 'ཞེ་ན', 'ན' : 'ཞེ་ན', 'མ' : 'ཞེ་ན', 'འ' : 'ཞེ་ན', 'ར' : 'ཞེ་ན', 'ལ' : 'ཞེ་ན', 'ས' : 'ཤེ་ན', 'མཐའ་མེད' : 'ཞེ་ན', 'ད་དྲག' : 'ཅེ་ན'}
        cig = {'ག' : 'ཅིག', 'ད' : 'ཅིག', 'བ' : 'ཅིག', 'ང' : 'ཞིག', 'ན' : 'ཞིག', 'མ' : 'ཞིག', 'འ' : 'ཞིག', 'ར' : 'ཞིག', 'ལ' : 'ཞིག', 'ས' : 'ཤིག', 'མཐའ་མེད' : 'ཞིག', 'ད་དྲག' : 'ཅིག', }
        # mostly for modern spoken Tibetan. in accord with Esukhia’s decision to make the agreement for this "new" particle
        gin = {'ད' : 'ཀྱིན', 'བ' : 'ཀྱིན', 'ས' : 'ཀྱིན', 'ག' : 'གིན', 'ང' : 'གིན', 'ན' : 'གྱིན', 'མ' : 'གྱིན', 'ར' : 'གྱིན', 'ལ' : 'གྱིན', 'ད་དྲག' : 'ཀྱིན'}
        cases = [(["གི", "ཀྱི", "གྱི", "ཡི"], dreldra), (["གིས", "ཀྱིས", "གྱིས", "ཡིས"], jedra), (["སུ", "ཏུ", "དུ", "རུ"] , ladon), (["སྟེ", "ཏེ", "དེ"], lhakce), (["ཀྱང", "ཡང", "འང"], gyendu), (["གམ", "ངམ", "དམ", "ནམ", "བམ", "མམ", "འམ", "རམ", "ལམ", "སམ", "ཏམ"], jedu), (["པ", "པོ", "བ", "བོ"], dagdra), (["གོ", "ངོ", "དོ", "ནོ", "བོ", "མོ", "འོ", "རོ", "ལོ", "སོ", "ཏོ"] , lardu), (["ཅིང",  "ཤིང", "ཞིང"], cing), (["ཅེས",  "ཞེས"], ces), (["ཅེའོ",  "ཤེའོ",  "ཞེའོ"], ceo), (["ཅེ་ན",  "ཤེ་ན",  "ཞེ་ན"], cena), (["ཅིག",  "ཤིག",  "ཞིག"], cig), (['ཀྱིན', 'གིན', 'གྱིན'], gin)]
        correction = ''
        for case in cases:
            if particle in case[0]:
                correction = case[1][final]
        return correction
    else:
        return '*'+particle

In [14]:
syl = 'རེག'
part = 'ཞིག'
previous_syl = syl_info(syl)
print(get_mingzhi(syl))
print(previous_syl)
print(part_agreement(previous_syl, part))

ར
རེག
ཅིག


In [8]:
'''
གམ—ཕྲད་ཚིག:རྒྱུན་སྤྱོད།/མིང་ཚིག:
གོ—ཕྲད་ཚིག:རྒྱུན་སྤྱོད།/བྱ་ཚིག:བྱ་ཚིག_###+གཟུགས་མི་འགྱུར་བ_ཐ་དད་པ/མིང་ཚིག:/ཚབ་ཚིག:རྒྱུན་སྤྱོད།
གྱི—ཕྲད་ཚིག:རྒྱུན་སྤྱོད།/མིང་ཚིག:བརྡ་རྙིང་།
གྱིས—ཕྲད་ཚིག:རྒྱུན་སྤྱོད།/བྱ་ཚིག:བགྱིད_སྐུལ་ཚིག_ཐ་དད་པ
ངམ—ཕྲད་ཚིག:རྒྱུན་སྤྱོད/རྒྱན་ཚིག:/མིང་ཚིག:
ངོ—མིང་ཚིག:རྒྱུན་སྤྱོད།
དམ—ཕྲད་ཚིག:རྒྱུན་སྤྱོད།/རྒྱན་ཚིག:ལྡོག་ཚིག
དུ—ཕྲད་ཚིག:རྒྱུན་སྤྱོད།
དེ—ཕྲད་ཚིག:རྒྱུན་སྤྱོད།/ཚབ་ཚིག:རྒྱུན་སྤྱོད།
དོ—གྲངས་ཚིག:/མིང་ཚིག:རྒྱུན་སྤྱོད།/གྲོགས་ཚིག:
ནམ—ཚབ་ཚིག:རྒྱུན་སྤྱོད།/མིང་ཚིག:
བམ—ཕྲད་ཚིག:རྒྱུན་སྤྱོད།/མིང་ཚིག:/རྒྱན་ཚིག:
མོ—མིང་ཚིག:ལྡོག་ཚིག_ལྡོག་ཚིག_ཆོས་ལུགས།_སྒྲ་རིག་པ།/གྲོགས་ཚིག:རྒྱུན་སྤྱོད།
ཞིག—གྲོགས་ཚིག:/བྱ་ཚིག:འཇིག_འདས་པ།_ཐ་མི་དད་པ
ཞིང—གྲོགས་ཚིག:/མིང་ཚིག:
འོ—ཕྲད་ཚིག:རྒྱུན་སྤྱོད།_བརྡ་རྙིང་།/འབོད་ཚིག:/མིང་ཚིག:རྒྱུན་སྤྱོད།
ཡང—ཕྲད་ཚིག:རྒྱུན་སྤྱོད།/རྒྱན་ཚིག:ལྡོག་ཚིག_ལྡོག་ཚིག
རུ—ཕྲད་ཚིག:རྒྱུན་སྤྱོད།/མིང་ཚིག:བརྡ་རྙིང་།
རོ—མིང་ཚིག:རྒྱུན་སྤྱོད།_ཆོས་ལུགས།
ལམ—མིང་ཚིག:རྒྱུན་སྤྱོད།_མིང་གི་རྣམ་གྲངས།_ཆོས་ལུགས།
ལོ—མིང་ཚིག:རྒྱུན་སྤྱོད།/གྲོགས་ཚིག:
ཤིང—མིང་ཚིག:རྒྱུན་སྤྱོད།/བྱ་ཚིག:བྱ་ཚིག_ཐ་མི་དད་པ
སུ—ཕྲད་ཚིག:རྒྱུན་སྤྱོད།/ཚབ་ཚིག:རྒྱུན་སྤྱོད།
སོ—མིང་ཚིག:རྒྱུན་སྤྱོད།_མངོན་བརྗོད།/ཚབ་ཚིག:རྒྱུན་སྤྱོད།
པ—མིང་ཚིག:རྒྱུན་སྤྱོད།/གྲོགས་ཚིག:
པོ—ཕྲད་ཚིག:རྒྱུན་སྤྱོད།/གྲོགས་ཚིག:/རྒྱན་ཚིག:
བ—མིང་ཚིག:རྒྱུན་སྤྱོད།
བོ—ཕྲད་ཚིག:རྒྱུན་སྤྱོད།/གྲོགས་ཚིག:
ཡི—ཕྲད་ཚིག:རྒྱུན་སྤྱོད།/མིང་ཚིག:
'''
# ambiguous particles : entries in Monlam1 that have other tags than གྲོགས་ཚིག་ or ཕྲད་ཚིག་
# དུ་ entry as question word/adverb missing in Monlam. I have added it in the list of ambiguous particles
# དུ་ | ༡ཕྲད་ཚིག (…)  6. གྲངས་ཅི་ཙམ་མམ་ག་ཚོད་ཅེས་པའི་དོན་སྟོན་བྱེད་དེ།      དཔེར་ན། ཁྱོད་ལ་རོགས་པ་དུ་ཡོད། གནས་ཚུལ་འདི་ཧ་གོ་མཁན་མི་དུ་ཡོད་ལྟ་བུ། 
non_ambiguous = ['གི', 'ཀྱི', 'གིས', 'ཀྱིས', 'ཡིས', 'ཏུ', 'སྟེ', 'ཏེ', 'ཀྱང', 'འང', 'མམ', 'འམ', 'སམ', 'ཏམ', 'ནོ', 'ཏོ', 'ཅིང', 'ཅེས', 'ཅེའོ', 'ཅིག', 'ཞེས', 'ཞེའོ', 'ཤེའོ', 'ཤིག', 'རམ']
ambiguous = ['གམ', 'གོ', 'གྱི', 'གྱིས', 'ངམ', 'ངོ', 'དམ', 'དུ', 'དེ', 'དོ', 'ནམ', 'བམ', 'མོ', 'ཞིག', 'ཞིང', 'འོ', 'ཡང', 'རུ', 'རོ', 'ལམ', 'ལོ', 'ཤིང', 'སུ', 'སོ', 'པ', 'པོ', 'བ', 'བོ', 'ཡི']

In [51]:
ambiguous = ['གམ', 'གོ', 'གྱི', 'གྱིས', 'ངམ', 'ངོ', 'དམ', 'དུ', 'དེ', 'དོ', 'ནམ', 'བམ', 'མོ', 'ཞིག', 'ཞིང', 'འོ', 'ཡང', 'རུ', 'རོ', 'ལམ', 'ལོ', 'ཤིང', 'སུ', 'སོ', 'ཡི']

syl_regex = r'[^་༌།༎༏༐༑༔ ]*[་༌།༎༏༐༑༔ ]*'
ambiguous_regex = {
'གམ' : r'[^་༌།༎༏༐༑༔ ]*[^ག]་གམ[་༌]', 
'གོ' : r'[^་༌།༎༏༐༑༔ ]*[^ག]་གོ[་༌]', 
'གྱི' : r'[^་༌།༎༏༐༑༔ ]*[^ནམརལ]་གྱི[་༌]', 
'གྱིས' : r'[^་༌།༎༏༐༑༔ ]*[^ནམརལ]་གྱིས[་༌]', 
'ངམ' : r'[^་༌།༎༏༐༑༔ ]*[^ང]་ངམ[་༌]', 
'ངོ' : r'[^་༌།༎༏༐༑༔ ]*[^ང]་ངོ[་༌]', 
'དམ' : r'[^་༌།༎༏༐༑༔ ]*[^ད]་དམ[་༌]', 
'དུ' : r'[^་༌།༎༏༐༑༔ ]*[^ངདནམརལ]་དུ[་༌]', 
'དེ' : r'[^་༌།༎༏༐༑༔ ]*[^ད]་དེ[་༌]', 
'དོ' : r'[^་༌།༎༏༐༑༔ ]*[^ད]་དོ[་༌]', 
'ནམ' : r'[^་༌།༎༏༐༑༔ ]*[^ན]་ནམ[་༌]', 
'བམ' : r'[^་༌།༎༏༐༑༔ ]*[^བ]་བམ[་༌]', 
'མོ' : r'[^་༌།༎༏༐༑༔ ]*[^མ]་མོ[་༌]', 
'ཞིག' : r'[^་༌།༎༏༐༑༔ ]*[^ངནམའརལ]་ཞིག[་༌]', 
'ཞིང' : r'[^་༌།༎༏༐༑༔ ]*[^ངནམའརལ]་ཞིང[་༌]', 
'ཡང' : r'[^་༌།༎༏༐༑༔ ]*[^ངནམའརལ]་ཡང[་༌]', 
'རུ' : r'[^་༌།༎༏༐༑༔ ]*[^རའ]་རུ[་༌]', 
'རོ' : r'[^་༌།༎༏༐༑༔ ]*[^ར]་རོ[་༌]', 
'ལམ' : r'[^་༌།༎༏༐༑༔ ]*[^ལ]་ལམ[་༌]', 
'ལོ' : r'[^་༌།༎༏༐༑༔ ]*[^ལ]་ལོ[་༌]', 
'ཤིང' : r'[^་༌།༎༏༐༑༔ ]*[^ས]་ཤིང[་༌]', 
'སུ' : r'[^་༌།༎༏༐༑༔ ]*[^ས]་སུ[་༌]', 
'སོ' : r'[^་༌།༎༏༐༑༔ ]*[^ས]་སོ[་༌]', 
'པ' : r'[^་༌།༎༏༐༑༔ ]*[^གདནབམས]་པ[་༌]', 
'པོ' : r'[^་༌།༎༏༐༑༔ ]*[^གདནབམས]་པོ[་༌]', 
'བ' : r'[^་༌།༎༏༐༑༔ ]*[^ངའརལ]་བ[་༌]', 
'བོ' : r'[^་༌།༎༏༐༑༔ ]*[^ངའརལ]་བོ[་༌]', 
'ཡི' : r'[^་༌།༎༏༐༑༔ ]*[^འ]་ཡི[་༌]', 
}

ambiguous_thame = ['ཞིག', 'ཞིང', 'འོ', 'ཡང', 'རུ', 'བ', 'བོ']
ambiguous_dadrag = ['གྱི', 'གྱིས', 'པ', 'པོ'] 

In [61]:
print('གམ གོ གྱི གྱིས ངམ ངོ དམ དུ དེ དོ ནམ བམ མོ ཞིག ཞིང འོ ཡང རུ རོ ལམ ལོ ཤིང སུ སོ པ པོ བ བོ ཡི')
left = 5
right = 4
possible_amb = {}
references = {}
for File in os.listdir('./texts'):
    print(File)
    with open('./texts/'+File, 'r', -1, 'utf-8-sig') as f:
        content = [line.strip() for line in f.readlines() if line.strip() != '']
    for part in ambiguous:
        print(part, end = ' ')
        amb = []
        refs = []
        for num, line in enumerate(content):
            if part in ambiguous_regex:
                regex = re.findall(syl_regex*left+ambiguous_regex[part]+syl_regex*right, line)
                if regex != []:
                    for elt in regex:
                        previous = elt.split('་'+part+'་')[0].split('་')
                        previous = previous[len(previous)-1]
                        if part in ambiguous_thame and syl_info(previous) != 'thame':
                            r = re.split('[་༌།༎༏༐༑༔ ]'+part+'[་༌།༎༏༐༑༔ ]', elt)
                            Tuple = (r[0]+'་', part+'་', r[1])
                            if Tuple not in amb:
                                amb.append(Tuple)
                                refs.append((File, num+1))
                        if part in ambiguous_dadrag and syl_info(previous) != 'dadrag':
                            r = re.split('[་༌།༎༏༐༑༔ ]'+part+'[་༌།༎༏༐༑༔ ]', elt)
                            Tuple = (r[0]+'་', part+'་', r[1])
                            if Tuple not in amb:
                                amb.append(Tuple)
                                refs.append((File, num+1))
                        if part not in ambiguous_thame and part not in ambiguous_dadrag:
                            r = re.split('[་༌།༎༏༐༑༔ ]'+part+'[་༌།༎༏༐༑༔ ]', elt)
                            Tuple = (r[0]+'་', part+'་', r[1])
                            if Tuple not in amb:
                                amb.append(Tuple)
                                refs.append((File, num+1))
        if amb != []:
            if part in possible_amb:
                possible_amb[part] = possible_amb[part] + amb
                references[part] = references[part] + refs
            else:
                possible_amb[part] = amb
                references[part] = refs

གམ གོ གྱི གྱིས ངམ ངོ དམ དུ དེ དོ ནམ བམ མོ ཞིག ཞིང འོ ཡང རུ རོ ལམ ལོ ཤིང སུ སོ པ པོ བ བོ ཡི
ཤེར་ཕྱིན། ཐ།.txt
གམ གོ གྱི གྱིས ངམ ངོ དམ དུ དེ དོ ནམ བམ མོ ཞིག ཞིང འོ ཡང རུ རོ ལམ ལོ ཤིང སུ སོ ཡི ཤེར་ཕྱིན། ཀ.txt
གམ གོ གྱི གྱིས ངམ ངོ དམ དུ དེ དོ ནམ བམ མོ ཞིག ཞིང འོ ཡང རུ རོ ལམ ལོ ཤིང སུ སོ ཡི ཤེར་ཕྱིན། མ།.txt
གམ གོ གྱི གྱིས ངམ ངོ དམ དུ དེ དོ ནམ བམ མོ ཞིག ཞིང འོ ཡང རུ རོ ལམ ལོ ཤིང སུ སོ ཡི བསྟོད་ཚོགས། ཀ.txt
གམ གོ གྱི གྱིས ངམ ངོ དམ དུ དེ དོ ནམ བམ མོ ཞིག ཞིང འོ ཡང རུ རོ ལམ ལོ ཤིང སུ སོ ཡི ཤེར་ཕྱིན། ཆ།.txt
གམ གོ གྱི གྱིས ངམ ངོ དམ དུ དེ དོ ནམ བམ མོ ཞིག ཞིང འོ ཡང རུ རོ ལམ ལོ ཤིང སུ སོ ཡི ཤེར་ཕྱིན། ཕ།.txt
གམ གོ གྱི གྱིས ངམ ངོ དམ དུ དེ དོ ནམ བམ མོ ཞིག ཞིང འོ ཡང རུ རོ ལམ ལོ ཤིང སུ སོ ཡི ཤེར་ཕྱིན། ཇ།.txt
གམ གོ གྱི གྱིས ངམ ངོ དམ དུ དེ དོ ནམ བམ མོ ཞིག ཞིང འོ ཡང རུ རོ ལམ ལོ ཤིང སུ སོ ཡི ཤེར་ཕྱིན། ག.txt
གམ གོ གྱི གྱིས ངམ ངོ དམ དུ དེ དོ ནམ བམ མོ ཞིག ཞིང འོ ཡང རུ རོ ལམ ལོ ཤིང སུ སོ ཡི ཤེར་ཕྱིན། ང་།.txt
གམ གོ གྱི གྱིས ངམ ངོ དམ དུ དེ དོ ནམ བམ མོ ཞིག ཞིང འོ ཡང རུ རོ ལམ ལོ ཤིང སུ སོ ཡི ཤེར་ཕྱིན། ཁ།.txt
གམ གོ གྱི 

In [62]:
for part in list(possible_amb):
    with open(part+'_concordance.csv', 'w', -1, 'utf-8-sig') as f:
        for num, possible in enumerate(possible_amb[part]):
            f.write(possible[0]+','+possible[1]+','+possible[2]+','+str(references[part][num][1])+','+references[part][num][0]+'\n')